In [93]:
import pickle
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier

In [111]:
with open('Finish_model.pkl', 'rb') as handle:
    b = pickle.load(handle)

In [100]:
data = [26, 88.39, 3, 0, 1, 1, 2.5, 0.7174, 0.6806, 23.0, 0.956522, 0.0]
# df = pd.DataFrame(data, columns=['runner_rank', 'Season', 'Metro_area', 'WL_SO', 'Entry_fee', 'PPM', 'Age_Rank' \
#                    , 'Gender_Rank', 'Total_races', 'Success_rate', 'success_metric', 'gender_F'])
# # df['Age'] = 50
# # df['gender_F'] = 1
# # df['runner_rank'] = 80.
# # df['Season'] = 2
# # df['Metro_area'] = 1
# # df['PPM'] = 2.5
# # df['Age_rank'] = 0.76
# # df['Gender_rank'] = 0.83
# # df['success_metric'] = 14
# # df['Total_races'] = 15
# # df['Success_rate'] = 0.85
# # df['Entry_fee'] = 1
# df

In [101]:
probabilities = b.predict_proba(data)

/Users/stef/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Number of features of the model must match the input. Model n_features is 14 and input n_features is 12 

In [97]:
probabilities

array([[ 0.10701007,  0.89298993]])

In [107]:
with open('DNS_model.pkl', 'rb') as handle:
    c = pickle.load(handle)

In [108]:
data2 = [42, 95.08, 2, 0, 1, 1, 2.70, 0.8150, 0.7306, 6.0, 1.0, 6.0,  1.0]

In [109]:
probability = c.predict_proba(data2)

/Users/stef/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [110]:
probability

array([[ 0.94211018,  0.05788982]])